In [1]:
import glob
import pandas as pd
import numpy as np
# import scipy.sparse 
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import re
import time

In [2]:
# path = './hw1_git/11775-hws/videos/*.mp4'
path = './hw1_git/11775-hws/asrs/*.txt'

filelist = []

for file in glob.glob(path):
    filelist.append(file)

In [3]:
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result

In [4]:
text = []
for i in range(len(filelist)):
# for i in range(10):
    with open (filelist[i], "r") as myfile:
        data=myfile.readlines()
        data=concatenate_list_data(data)
    text.append(data)
    if i % 1000 == 0:
        print('{}th txt file reading...'.format(i))
    else: pass

0th txt file reading...
1000th txt file reading...
2000th txt file reading...


#### Vectorizer TDIF Vectorizer

In [23]:
##TF-IDF Vectorization
vectorize = TfidfVectorizer(stop_words="english",min_df=2,sublinear_tf=True)#, norm='l1')    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
norm_bow = vectorize.fit_transform(text).toarray()
norm_data = pd.DataFrame(norm_bow)
norm_data.head(3)
# bow = vectorize.fit_transform(text)#.toarray()

,0,1,2,3,4,5,6,7,8,9,...,3522,3523,3524,3525,3526,3527,3528,3529,3530,3531
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
vect = CountVectorizer(stop_words="english")
# vect = CountVectorizer()
bow = vect.fit_transform(text).toarray()
norm_bow = normalize(bow, norm = 'l1', axis=1)
norm_data = pd.DataFrame(norm_bow)
norm_data.head(3)

,0,1,2,3,4,5,6,7,8,9,...,6976,6977,6978,6979,6980,6981,6982,6983,6984,6985
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### ASRS Modeling...

In [6]:
# Making Video name
video_name_ind = []
for i in range(len(filelist)):
    match_front = re.search('asrs/', filelist[i])
    match_end = re.search('.txt', filelist[i])
    video_name_ind.append(filelist[i][match_front.end():match_end.start()])
    video_name = pd.DataFrame({'video': video_name_ind})
#     print(i, filelist[i][match_front.end():match_end.start()])


In [7]:
# Making features columns
k = norm_data.shape[1]
column_name = ['video']
for i in range(k):
    column_name.append('feature_{}'.format(i))

total_data = pd.concat([video_name, norm_data], axis = 1)
total_data.columns = column_name

In [8]:
total_data.head(3)

,video,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_6976,feature_6977,feature_6978,feature_6979,feature_6980,feature_6981,feature_6982,feature_6983,feature_6984,feature_6985
0,HVC3995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HVC3694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HVC3332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
train_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/train', sep = ' ', header = None)
valid_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/val', sep = ' ', header = None)
test_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/test.video', sep = ' ', header = None)

train_ind['Data'] = 'TRAIN'
valid_ind['Data'] = 'VALID'
test_ind[1] = 'UNK'
test_ind['Data'] = 'TEST'

train_ind.columns = ['video','target','Data']
valid_ind.columns = ['video','target','Data']
test_ind.columns = ['video','target','Data']

In [10]:
data_lable = pd.concat([train_ind, valid_ind, test_ind], axis = 0).reset_index().drop('index', axis = 1)
# data_lable['target_p001'] = 
data_lable['target_p001'] = data_lable['target']
data_lable['target_p002'] = data_lable['target']
data_lable['target_p003'] = data_lable['target']
data_lable['target_p001_10'] = 1
data_lable['target_p002_10'] = 1
data_lable['target_p003_10'] = 1

data_lable['target_p001'][data_lable['target'] != 'P001'] = 'Other'
data_lable['target_p002'][data_lable['target'] != 'P002'] = 'Other'
data_lable['target_p003'][data_lable['target'] != 'P003'] = 'Other'
data_lable['target_p001_10'][data_lable['target'] != 'P001'] = 0
data_lable['target_p002_10'][data_lable['target'] != 'P002'] = 0
data_lable['target_p003_10'][data_lable['target'] != 'P003'] = 0

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/di

In [11]:
total_mart = total_data.merge(data_lable, how = 'right', on = 'video')
total_mart = total_mart.fillna(0)

In [12]:
train_mart = total_mart[total_mart['Data'] == 'TRAIN']
valid_mart = total_mart[total_mart['Data'] == 'VALID']
test_mart  = total_mart[total_mart['Data'] == 'TEST']

In [13]:
print(train_mart.shape, valid_mart.shape, test_mart.shape)

((836, 6995), (400, 6995), (1699, 6995))


#### Modeling....

In [14]:
train_mart.head(3)

,video,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_6984,feature_6985,target,Data,target_p001,target_p002,target_p003,target_p001_10,target_p002_10,target_p003_10
1,HVC3694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,TRAIN,Other,Other,Other,0,0,0
3,HVC711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,P002,TRAIN,Other,P002,Other,0,1,0
19,HVC3481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,TRAIN,Other,Other,Other,0,0,0


In [15]:
def modeling_ap_SVM(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = SVC(kernel=chi2_kernel, probability=True)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of SVM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_AdaB(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = AdaBoostClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of AdaBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_Boost(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = GradientBoostingClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of Boosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_xgb(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_lgbm(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = LGBMClassifier(n_erandom_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

In [16]:
k = norm_data.shape[1]
SVM_results_p001 = modeling_ap_SVM(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
SVM_results_p002 = modeling_ap_SVM(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
SVM_results_p003 = modeling_ap_SVM(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of SVM Modeling : 13.5096669197 seconds =====
0.05229870968993119
===== The time consuming of SVM Modeling : 13.4838149548 seconds =====
0.058798417072135276
===== The time consuming of SVM Modeling : 13.3538599014 seconds =====
0.4779058189539346


In [17]:
AdaB_results_p001 = modeling_ap_AdaB(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
AdaB_results_p002 = modeling_ap_AdaB(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
AdaB_results_p003 = modeling_ap_AdaB(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of AdaBoosting Modeling : 5.50566697121 seconds =====
0.06038898518597128
===== The time consuming of AdaBoosting Modeling : 5.53416013718 seconds =====
0.08541283242440459
===== The time consuming of AdaBoosting Modeling : 5.42733192444 seconds =====
0.11844260385818306


In [18]:
Boost_results_p001 = modeling_ap_Boost(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Boost_results_p002 = modeling_ap_Boost(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Boost_results_p003 = modeling_ap_Boost(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of Boosting Modeling : 8.39383792877 seconds =====
0.10199230216725215
===== The time consuming of Boosting Modeling : 8.57782292366 seconds =====
0.08413041617594157
===== The time consuming of Boosting Modeling : 7.9631228447 seconds =====
0.2032935070908536


In [19]:
Xgb_results_p001 = modeling_ap_xgb(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002 = modeling_ap_xgb(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Xgb_results_p003 = modeling_ap_xgb(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of XgBoosting Modeling : 13.110724926 seconds =====
0.07612313264975729
===== The time consuming of XgBoosting Modeling : 13.025233984 seconds =====
0.07148545613244141
===== The time consuming of XgBoosting Modeling : 13.126335144 seconds =====
0.3139753639029895


In [20]:
LGBM_results_p001 = modeling_ap_lgbm(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
LGBM_results_p002 = modeling_ap_lgbm(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
LGBM_results_p003 = modeling_ap_lgbm(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of XgBoosting Modeling : 6.77059006691 seconds =====
0.056751051826715734
===== The time consuming of XgBoosting Modeling : 6.78629589081 seconds =====
0.08525656893756202
===== The time consuming of XgBoosting Modeling : 6.77073502541 seconds =====
0.12622753023391728


===== The time consuming of SVM Modeling : 6.78026795387 seconds =====
0.037813639154038584
===== The time consuming of SVM Modeling : 6.6843829155 seconds =====
0.042515897441134584
===== The time consuming of SVM Modeling : 6.79400086403 seconds =====
0.23016056095521928

===== The time consuming of AdaBoosting Modeling : 2.99216413498 seconds =====
0.07452626524617899
===== The time consuming of AdaBoosting Modeling : 2.99656176567 seconds =====
0.08403003528707245
===== The time consuming of AdaBoosting Modeling : 2.99476003647 seconds =====
0.09219943362419089

===== The time consuming of Boosting Modeling : 4.99071788788 seconds =====
0.08310967054057486
===== The time consuming of Boosting Modeling : 4.99086213112 seconds =====
0.08277607559261606
===== The time consuming of Boosting Modeling : 4.72514605522 seconds =====
0.2678300605904397

===== The time consuming of XgBoosting Modeling : 6.4967110157 seconds =====
0.09871126378293811
===== The time consuming of XgBoosting Modeling : 6.55879688263 seconds =====
0.06946966259440435
===== The time consuming of XgBoosting Modeling : 6.62962818146 seconds =====
0.308958578141431